In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score
import numpy as np
import nltk
from nltk.corpus import wordnet
import random
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Layer
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Lambda
from sklearn.utils.class_weight import compute_class_weight

# Note: This notebook is designed to run in Google Colab.
# Data paths refer to Google Drive mount points.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_train = pd.read_csv('/content/sample_data/multilingual_hate_speech_test.csv', encoding='utf-8')
print(df_train['label'].value_counts())
df_test = pd.read_csv('/content/sample_data/combined_test_dataset.csv', encoding='utf-8')

train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=42)

train_texts = train_df['text'].values
train_labels = train_df['label'].apply(lambda x: 1 if x == 'HOF' else 0).values
val_texts = val_df['text'].values
val_labels = val_df['label'].apply(lambda x: 1 if x == 'HOF' else 0).values

class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

print(f"Class weights: {class_weight_dict}")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/LaBSE')
labse_model = TFAutoModel.from_pretrained('sentence-transformers/LaBSE')

def set_trainable_layers(model, num_trainable_layers):
    for layer in model.layers:
        layer.trainable = False

    # Unfreezed the top layers (including the classifier head)
    for layer in model.layers[-num_trainable_layers:]:
        layer.trainable = True

# unfreezed last 8 layers
set_trainable_layers(labse_model, num_trainable_layers=8)
print("\nTrainable Layers Status:")
for i, layer in enumerate(labse_model.layers[-10:]):
    print(f"Layer {len(labse_model.layers)-10+i}: {layer.name} -> {layer.trainable}")

def preprocess_data(texts, tokenizer, max_length=128):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='tf'
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])
    return tf.concat(input_ids, axis=0), tf.concat(attention_masks, axis=0)

train_input_ids, train_attention_mask = preprocess_data(train_texts, tokenizer)
val_input_ids, val_attention_mask = preprocess_data(val_texts, tokenizer)

class F1ScoreCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data):
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_targ = self.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict)
        print(f" --- val_f1: {_val_f1}")
        logs['val_f1'] = _val_f1

def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.keras.backend.clip(y_pred, epsilon, 1. - epsilon)
        pt = tf.where(tf.keras.backend.equal(y_true, 1), y_pred, 1 - y_pred)
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1. - pt, gamma) * tf.keras.backend.log(pt))
    return focal_loss_fixed

class LaBSEWrapper(tf.keras.layers.Layer):
    def __init__(self, labse_model):
        super(LaBSEWrapper, self).__init__()
        self.labse_model = labse_model

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.labse_model(input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state

input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32)
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32)

labse_wrapper = LaBSEWrapper(labse_model)
embeddings = labse_wrapper([input_ids, attention_mask])
cls_embedding = embeddings[:, 0, :]

hidden_layer = tf.keras.layers.Dense(250, activation='relu', kernel_regularizer=regularizers.l2(0.01))(cls_embedding)
dropout_layer = tf.keras.layers.Dropout(0.4)(hidden_layer)
hidden_layer2 = tf.keras.layers.Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.01))(dropout_layer)
output = tf.keras.layers.Dense(1, activation='sigmoid')(hidden_layer2)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

embedding_model = tf.keras.Model(
    inputs=[input_ids, attention_mask],
    outputs=cls_embedding
)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=focal_loss(),
              metrics=['accuracy'])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

f1_callback = F1ScoreCallback(validation_data=([val_input_ids, val_attention_mask], val_labels))

history = model.fit(
    [train_input_ids, train_attention_mask],
    train_labels,
    epochs=70,
    batch_size=32,
    validation_data=([val_input_ids, val_attention_mask], val_labels),
    callbacks=[f1_callback, early_stopping],
    class_weight=class_weight_dict
)

In [ ]:
val_predictions_prob = model.predict([val_input_ids, val_attention_mask]).flatten()
thresholds = np.arange(0.1, 1.0, 0.1)
f1_scores = []
for threshold in thresholds:
    binary_predictions = (val_predictions_prob >= threshold).astype(int)
    f1 = f1_score(val_labels, binary_predictions)
    f1_scores.append(f1)

best_threshold_index = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_index]
print(f"\nBest Threshold: {best_threshold:.2f} with F1 Score: {f1_scores[best_threshold_index]:.4f}")

val_predictions = (val_predictions_prob >= best_threshold).astype(int)
val_accuracy = accuracy_score(val_labels, val_predictions)
val_loss = tf.keras.losses.binary_crossentropy(val_labels, val_predictions_prob).numpy().mean()
print(f"Validation Accuracy (Best Threshold): {val_accuracy:.4f}")
print(f"Validation Loss (Best Threshold): {val_loss:.4f}")

In [ ]:
test_texts = df_test['text'].values
test_labels = df_test['label'].apply(lambda x: 1 if x == 'HOF' else 0).values
test_input_ids, test_attention_mask = preprocess_data(test_texts, tokenizer)

test_predictions_prob = model.predict([test_input_ids, test_attention_mask]).flatten()
test_predictions = (test_predictions_prob >= best_threshold).astype(int)

f1 = f1_score(test_labels, test_predictions)
test_accuracy = accuracy_score(test_labels, test_predictions)

print("\nClassification Report (Test):")
print(classification_report(test_labels, test_predictions, target_names=['NOT', 'HOF']))
print(f"F1 Score (Test): {f1:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
test_texts = df_test['text'].values
test_labels = df_test['label'].apply(lambda x: 1 if x == 'HOF' else 0).values
test_input_ids, test_attention_mask = preprocess_data(test_texts, tokenizer)

test_predictions_prob = model.predict([test_input_ids, test_attention_mask]).flatten()
test_predictions = (test_predictions_prob >= best_threshold).astype(int)

f1_micro = f1_score(test_labels, test_predictions, average='micro')
f1_macro = f1_score(test_labels, test_predictions, average='macro')
test_accuracy = accuracy_score(test_labels, test_predictions)

print("\nClassification Report (Test):")
print(classification_report(test_labels, test_predictions, target_names=['NOT', 'HOF']))
print(f"F1 Score (Micro - Test): {f1_micro:.4f}")
print(f"F1 Score (Macro - Test): {f1_macro:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
test_samples = [
    "This team is amazing!",                     # Should be NOT (0)
    "I hate you, you're worthless!",             # Should be HOF (1)
    "Let's discuss the project tomorrow",        # Neutral (0)
    "People like you should die",                # Extreme hate (1)
    "What an idiot!",                            # Insult (likely 1)
    "The weather is nice today",                  # Neutral (0)

    "هذا الفريق رائع جدًا!",                    # Should be NOT (0)
    "أكرهك، أنت لا تساوي شيئًا!",               # Should be HOF (1)
    "دعنا نناقش المشروع غدًا",                  # Neutral (0)
    "أمثالك يجب أن يُقتَلوا",                   # Extreme hate (1)
    "يا غبي!",                                  # Insult (likely 1)
    "الجو جميل اليوم",                          # Neutral (0)

    "Cette équipe est incroyable !",             # Should be NOT (0)
    "Je te déteste, tu ne vaux rien !",         # Should be HOF (1)
    "On parle du projet demain ?",              # Neutral (0)
    "Des gens comme toi méritent de mourir",    # Extreme hate (1)
    "Quel imbécile !",                          # Insult (likely 1)
    "Il fait beau aujourd'hui",                 # Neutral (0)

    "أنت عار على المجتمع",                     # Hate (1)
    "كلنا بشر ويجب أن نحترم بعضنا البعض",      # Positive (0)
    "أنت مجرد نكرة لا قيمة لك",                # Insult (1)
    "سأذهب للتسوق في المساء",                  # Neutral (0)
    "لا مكان لأمثالك بيننا",                   # Hate (1)
    "مباراة اليوم كانت ممتعة جدًا",            # Positive (0)

    "Tu es une honte pour l'humanité",          # Hate (1)
    "Tout le monde mérite du respect",          # Positive (0)
    "T'es qu'un bon à rien",                    # Insult (1)
    "Je vais cuisiner ce soir",                 # Neutral (0)
    "On ne veut pas de toi ici",                # Hate (1)
    "Ce film était vraiment génial",            # Positive (0)
]

print("\n" + "="*50)
print("MODEL VALIDATION TEST (REAL EXAMPLES)")
print("="*50)

for text in test_samples:
    # Preprocess
    inputs = tokenizer(
        text,
        return_tensors="tf",
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    prob = model.predict([inputs["input_ids"], inputs["attention_mask"]])[0][0]
    prediction = "HOF" if prob >= best_threshold else "NOT"

    color = "\033[91m" if prediction == "HOF" else "\033[92m"
    print(f"{color}Text: {text[:50]}{'...' if len(text)>50 else ''}")
    print(f"Prediction: {prediction} ({prob:.4f})\033[0m")

In [ ]:
# Save and export
export_path = "/content/drive/MyDrive/WorkSphere_labse_Model_best"

@tf.function(input_signature=[
    tf.TensorSpec(shape=[None, 128], dtype=tf.int32, name="input_ids"),
    tf.TensorSpec(shape=[None, 128], dtype=tf.int32, name="attention_mask")
])
def serve(input_ids, attention_mask):
    return {"prediction": model([input_ids, attention_mask])}

tf.saved_model.save(
    model,
    export_path,
    signatures={
        'serving_default': serve.get_concrete_function(
            input_ids=tf.TensorSpec(shape=[None, 128], dtype=tf.int32),
            attention_mask=tf.TensorSpec(shape=[None, 128], dtype=tf.int32)
        )
    }
)

tokenizer.save_pretrained(export_path + "_tokenizer")

In [ ]:
loaded_model = tf.saved_model.load(export_path)
infer = loaded_model.signatures['serving_default']

test_text = "This is a test sentence"
inputs = tokenizer(test_text, return_tensors='tf', padding='max_length', truncation=True, max_length=128)

output = infer(
    input_ids=inputs['input_ids'],
    attention_mask=inputs['attention_mask']
)
print(f"Input: '{test_text}'\nPrediction: {output['prediction'].numpy()[0][0]:.4f}")

In [ ]:
def evaluate_prediction(text):
    inputs = tokenizer(text,
                     return_tensors='tf',
                     padding='max_length',
                     truncation=True,
                     max_length=128)

    output = infer(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask']
    )

    pred = output['prediction'].numpy()[0][0]
    interpretation = "Hateful" if pred > 0.7 else "Safe" if pred < 0.3 else "Uncertain"

    print(f"""
    Text: {text}
    Raw score: {pred:.4f}
    Interpretation: {interpretation}
    Confidence: {'High' if abs(pred-0.5)>0.4 else 'Medium' if abs(pred-0.5)>0.2 else 'Low'}
    """)


evaluate_prediction("You're amazing!")
evaluate_prediction("I hate everyone!")
evaluate_prediction("This is a test")

In [ ]:
embedding_model = tf.keras.Model(
    inputs=[input_ids, attention_mask],
    outputs=cls_embedding
)

@tf.function(input_signature=[
    tf.TensorSpec(shape=[None, 128], dtype=tf.int32, name='input_word_ids'),
    tf.TensorSpec(shape=[None, 128], dtype=tf.int32, name='input_mask')
])
def serving_fn(input_word_ids, input_mask):
    return embedding_model([input_word_ids, input_mask])

tf.saved_model.save(
    embedding_model,
    '/content/drive/MyDrive/eng_models_att2/eng_labse_hate_embeddings',
    signatures={
        'serving_default': serving_fn.get_concrete_function()
    }
)


In [ ]:
# loading verification
loaded_model = tf.saved_model.load('/content/drive/MyDrive/eng_labse_hate_embeddings')
serving_fn = loaded_model.signatures['serving_default']

In [ ]:
# Test with dummy data
test_input = {
    'input_word_ids': tf.constant([[1]*128], dtype=tf.int32),
    'input_mask': tf.constant([[1]*128], dtype=tf.int32)
}
embeddings = serving_fn(**test_input)['output_0']
print(embeddings.shape)  #  (1, 768)